In [5]:
import pandas as pd
import random

In [4]:
data = pd.read_csv('data/train.csv')
data.head()


,id,time,x,y
0,1,0,800,0
1,1,780,780,0
2,1,1572,792,0
3,1,2392,820,0
4,1,3196,804,0


In [52]:
test = pd.DataFrame(columns=data.columns)
train = pd.DataFrame(columns=data.columns)
PART = 0.1
is_equal = False
ids = data.id.unique().tolist()
while not is_equal:
    test_ids = random.sample(list(ids), int(PART * len(ids)))
    train_ids = set(ids) - set(test_ids)

    test = data.loc[data.id.isin(test_ids)]
    train = data.loc[data.id.isin(train_ids)]

    if 0.99 < (test.x.mean() / train.x.mean()) < 1.01:
        is_equal = True

In [53]:
test.x.mean(), train.x.mean()

(613.4280319535221, 618.3239617924704)

In [54]:
test.to_csv('data/split_data/test.csv')
train.to_csv('data/split_data/train.csv')

In [15]:
import pandas as pd

train = pd.read_csv('data/split_data/train.csv', index_col=0)
test = pd.read_csv('data/split_data/test.csv', index_col=0)

In [19]:
PREDICT_PART = 10

from main_features.rr_features import StatFeatures

name_cols = ['tension_index_int',
             'mode_int',
             'std_int',
             'mean_int',
             'var_int',
             'pnn50_int',
             'RMSSD_int',
             'ivr_int',
             'vpr_int',
             'papr_int',
             'tension_index_user',
             'mode_user',
             'std_user',
             'mean_user',
             'var_user',
             'pnn50_user',
             'RMSSD_user',
             'ivr_user',
             'vpr_user',
             'papr_user'] + \
            [f'r-r-{i}' for i in range(PREDICT_PART)] + \
            [f'y{i}' for i in range(PREDICT_PART)]
df_train_stats = pd.DataFrame(columns=name_cols)
df_test_stats = pd.DataFrame(columns=name_cols)
df_train_stats.head()

,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9


In [20]:
from tqdm import tqdm

max_id = train.id.unique().max()
for id_user in train.id.unique():
    print(f'ID {id_user} из {max_id}')
    stats_for_id = StatFeatures(train.loc[train.id == id_user, 'x'], count_intervals=PREDICT_PART).get_statistic()
    for i in tqdm(range(train.loc[train.id == id_user].shape[0] - PREDICT_PART)):
        if train.iloc[i: i + PREDICT_PART, 2].max() == train.iloc[i: i + PREDICT_PART, 2].min():
            continue
        stats_for_interval = StatFeatures(train.iloc[i: i + PREDICT_PART, 2], count_intervals=PREDICT_PART).get_statistic()
        df_train_stats.loc[i, :] = stats_for_interval + stats_for_id + list(train.iloc[i: i + PREDICT_PART, 2].values) + list(
            train.iloc[i: i + PREDICT_PART, 3].values)

df_train_stats

ID 1 из 275


  5%|▍         | 10/216 [00:00<00:02, 99.24it/s]

ID 2 из 275


 21%|██        | 20/95 [00:00<00:00, 96.50it/s]

ID 3 из 275


...

ID 273 из 275


 11%|█         | 20/190 [00:00<00:01, 95.48it/s]

ID 275 из 275


100%|██████████| 190/190 [00:02<00:00, 94.77it/s]


,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9
0,71.0429,782,72.5216,858.4,8.44846,33.3333,79.1398,0.00111111,7.10429e-06,0.000255754,...,0,0,0,0,0,0,0,0,0,0
1,71.0732,781.667,70.1554,862,8.13868,33.3333,75.0822,0.00111111,7.10732e-06,0.000255864,...,0,0,0,0,0,0,0,0,0,0
2,112.38,794.5,68.6841,863.2,7.95692,44.4444,85.9457,0.00178571,7.49198e-06,0.000377596,...,0,0,0,0,0,0,0,0,0,0
3,69.8858,777.667,70.6843,861.6,8.20384,55.5556,88.1514,0.00108696,6.98858e-06,0.00025718,...,0,0,0,0,0,0,0,0,0,0
4,53.5886,833.067,80.4885,853.2,9.43372,77.7778,94.6854,0.000892857,5.35886e-06,0.000240077,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3646,36.366,616,292.175,554.4,52.7011,88.8889,428.652,0.000448029,1.45464e-06,0.000811688,...,1,0,0,0,0,0,0,0,0,0
3647,29.1767,614.229,304.26,520.4,58.4665,100,506.835,0.000358423,1.45883e-06,0.000651223,...,0,0,0,0,0,0,0,0,0,0
3648,21.9967,611.04,306.662,494.4,62.0271,88.8889,392.247,0.000268817,1.46645e-06,0.000490966,...,0,0,0,0,0,0,0,0,0,0
3649,21.9967,611.04,306.662,494.4,62.0271,100,507.061,0.000268817,1.46645e-06,0.000490966,...,0,0,0,0,0,0,0,0,0,0


In [21]:
max_id = test.id.unique().max()
for id_user in test.id.unique():
    print(f'ID {id_user} из {max_id}')
    stats_for_id = StatFeatures(test.loc[test.id == id_user, 'x'], count_intervals=PREDICT_PART).get_statistic()
    for i in tqdm(range(test.loc[test.id == id_user].shape[0] - PREDICT_PART)):
        if test.iloc[i: i + PREDICT_PART, 2].max() == test.iloc[i: i + PREDICT_PART, 2].min():
            continue
        stats_for_interval = StatFeatures(test.iloc[i: i + PREDICT_PART, 2], count_intervals=PREDICT_PART).get_statistic()
        df_test_stats.loc[i, :] = stats_for_interval + stats_for_id + list(test.iloc[i: i + PREDICT_PART, 2].values) + list(
            test.iloc[i: i + PREDICT_PART, 3].values)


  3%|▎         | 8/310 [00:00<00:04, 75.12it/s]

ID 14 из 252


  5%|▍         | 20/419 [00:00<00:04, 95.89it/s]

ID 19 из 252


  1%|          | 10/947 [00:00<00:09, 95.99it/s]

...

 14%|█▎        | 20/146 [00:00<00:01, 92.76it/s]

ID 251 из 252


 17%|█▋        | 10/60 [00:00<00:00, 92.14it/s]

ID 252 из 252


100%|██████████| 60/60 [00:00<00:00, 91.81it/s]


In [22]:
df_train_stats.loc[(df_train_stats.y0 == 1) |
                   (df_train_stats.y1 == 1) |
                    (df_train_stats.y2 == 1) |
                    (df_train_stats.y3 == 1) |
                    (df_train_stats.y4 == 1)]


,tension_index_int,mode_int,std_int,mean_int,var_int,pnn50_int,RMSSD_int,ivr_int,vpr_int,papr_int,...,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9
349,1736.11,720,4.69515,718.4,0.653557,0,6.79869,0.025,8.68056e-05,0.000555556,...,0,0,0,0,1,1,1,1,1,1
350,1748.41,714.933,4.29987,717.6,0.599202,0,6.1101,0.025,8.74207e-05,0.000559493,...,0,0,0,1,1,1,1,1,1,1
351,1748.41,714.933,4.71405,718,0.656552,0,7.7746,0.025,8.74207e-05,0.000559493,...,0,0,1,1,1,1,1,1,1,1
352,1163.76,716.073,6.26808,718.8,0.872021,0,10.4137,0.0166667,5.81878e-05,0.000558602,...,0,1,1,1,1,1,1,1,1,1
353,549.186,718.767,21.586,725.2,2.97656,22.2222,31.496,0.00789474,1.83062e-05,0.000834763,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3642,43.3698,623.85,247.463,613.6,40.3297,66.6667,389.879,0.000541126,1.73479e-06,0.000801475,...,1,1,1,1,1,0,0,0,0,0
3643,43.2043,622.2,292.739,565.2,51.7939,77.7778,421.091,0.000537634,1.44014e-06,0.00096432,...,1,1,1,1,0,0,0,0,0,0
3644,36.366,616,291.351,549.2,53.0501,77.7778,395.634,0.000448029,1.45464e-06,0.000811688,...,1,1,1,0,0,0,0,0,0,0
3645,36.366,616,292.09,554,52.7238,77.7778,427.027,0.000448029,1.45464e-06,0.000811688,...,1,1,0,0,0,0,0,0,0,0


In [23]:
df_test_stats.to_csv('data/split_data/test_stats.csv')
df_train_stats.to_csv('data/split_data/train_stats.csv')